In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer as si
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier

In [133]:
df = pd.read_csv("/Users/prajualpillai/Desktop/prajual/Personal_git/boosting/train.csv")

In [139]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [134]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [135]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [137]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [138]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [129]:
set(df["Embarked"])

{'C', 'Q', 'S', nan}

In [130]:
len(set(df["Ticket"])), len(df["Ticket"])

(681, 891)

In [100]:
# Convert categorical values to numeraical
# 1. One-hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
label_encoder = LabelEncoder()

In [141]:
imp_median = si(missing_values=np.nan, strategy="most_frequent")
imp_median.fit(df)
df2 = pd.DataFrame(imp_median.transform(df),columns=df.columns)

In [143]:
df2.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,B96 B98,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,B96 B98,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,B96 B98,S


In [149]:
df2.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [101]:
imp_median = si(missing_values=np.nan, strategy="median")
imp_median.fit(df[["Age"]])
df[["Age"]] = imp_median.transform(df[["Age"]])

In [102]:
df2 = df.drop(columns=["Cabin","Name"], axis=1)
df3 = df2.dropna(how="any", axis=0)

In [103]:
def encode(df_3):
    integer_encoded = label_encoder.fit_transform(df_3)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

In [ ]:
embarked_encoded = encode(df3["Embarked"])
sex_encoded = encode(df3["Sex"])
df3[["em_1","em_2", "em_3"]] = embarked_encoded
df3[["Sex_1","Sex_2"]] = sex_encoded
integer_encoded = label_encoder.fit_transform(df3["Ticket"])
df3["Ticket"] = integer_encoded.reshape(len(integer_encoded),1)

In [106]:
len(set(df3["Ticket"]))

680

In [112]:
independent_values = df3.drop(columns=["Survived", "Sex", "Embarked"], axis=1)
dependent_value = df3[["Survived"]]

In [113]:
independent_values.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
em_1           0
em_2           0
em_3           0
Sex_1          0
Sex_2          0
dtype: int64

In [114]:
independent_values.dtypes

PassengerId      int64
Pclass           int64
Age            float64
SibSp            int64
Parch            int64
Ticket           int64
Fare           float64
em_1           float64
em_2           float64
em_3           float64
Sex_1          float64
Sex_2          float64
dtype: object

In [115]:
gradient_booster = GradientBoostingClassifier(learning_rate=0.1)
gradient_booster.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'log_loss',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [116]:
kf = KFold(n_splits=5,random_state=42,shuffle=True)
for train_index,val_index in kf.split(independent_values):
    X_train,X_val = independent_values.iloc[train_index],independent_values.iloc[val_index],
    y_train,y_val = dependent_value.iloc[train_index],dependent_value.iloc[val_index],

In [117]:
gradient_booster.fit(X_train,y_train)

/opt/anaconda3/envs/amex/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier()

In [119]:
print(classification_report(y_val,gradient_booster.predict(X_val)))

              precision    recall  f1-score   support

           0       0.84      0.89      0.86       116
           1       0.76      0.67      0.71        61

    accuracy                           0.81       177
   macro avg       0.80      0.78      0.79       177
weighted avg       0.81      0.81      0.81       177



In [120]:
import pickle
with open('clf.pickle', 'wb') as f:
    pickle.dump(gradient_booster, f)